## Step 1: Prepare Documents

In [2]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
# Change the format of documents all topics are in the same level

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

## Step 2: Create Embeddings using Pretrained Models

"sentence_transformers" helps to call a pre-trained model and use it to create embedding very easy.
So it create a vector (numeric vector) from our input, and we cna save that vector to our dataset.

In [4]:
from sentence_transformers import SentenceTransformer
# If an error shows during importing, you need to do some steps in "demo_es.ipynb" file

model = SentenceTransformer("all-mpnet-base-v2")

/Users/hani/Documents/llm-zoomcamp/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model.encode("This is a simple sentences")
# the output is a vector

len(model.encode("This is a simple sentences"))
# different model has different length

768

In [6]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

AS we see from the documents[1], there are four sections availabke.
The second and fourth ones are categorical and do not have that much information.
Among the first, and third ones, we are going to use the first one ('text')

In [7]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

## Step 3: Setup ElasticSearch connection 

In [1]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '53db3e168796', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'OrHnNkhLQ-yRQaAS4opKwg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Step 4: Create Mappings and Index

* Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

* Each document is a collection of fields, which each have their own data type.

* We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored 

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [9]:
index_name = "course-questions"

# To make sure there is no index from previous round
es_client.indices.delete(index=index_name, ignore_unavailable=True)

# Create the index
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Step 5: Add documents into index

In [10]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  ## Step 6: Create end user query

In [11]:
search_term = "windows or mac?"

# convert the query to vector using encoder (model)
vector_search_term = model.encode(search_term)

In [12]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [13]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]
# The results shows the top five answers sorted based pn score which is between 0 and 1.
# In this case the score of highest one is 0.71 which is a decent score.
# And shows the relevence of answer to the query

[{'_index': 'course-questions',
  '_id': 'MPiOn5AB5wtEKsWV8dFg',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'Q_iOn5AB5wtEKsWV_dSe',
  '_score': 0.6134736,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

We just saw that the query is just "windows or mac" but the semantic search understand the underlaying concept, so the answer was align well with the purpose of the question.

## Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [15]:
# Apply filter on the search domain (e.g to inlcude search not in the whole
# documnets, but in one of the course)

knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [22]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
      # "match": {"course": "data-engineering-zoomcamp"}
    },
    knn=knn_query,
    size=5
    # If it is True, it provide more info about the method of score calculation
    # explain = True
)

In [23]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'MPiOn5AB5wtEKsWV8dFg',
  '_score': 1.4937057,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965495198965073,
    -0.0006259257206693292,
    -0.016629504039883614,
    0.05285143107175827,
    0.05476531386375427,
    -0.03133983165025711,
    0.029942642897367477,
    -0.04808562621474266,
    0.04467548429965973,
    0.005839455407112837,
    0.016233131289482117,
    0.012001212686300278,
    -0.0312223918735981,
    0.01660064235329628,
    -0.04886900633573532,
    -0.06496303528547287,
    0.046434178948402405,
    -0.009297648444771767,
    -0.0642528235912323,
    -0.01373270619660616,
    -0.015976140275597572,
    0.00862955953925848,
    -0.0244790073484182